In [33]:
# requirements 
%pip install pandas --quiet 
%pip install flask requests --quiet
%pip install ollama --quiet

print("Requirements installed successfully!")


You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Requirements installed successfully!


In [41]:
# imports
print("Starting imports...")
import pandas as pd
from flask import Flask, request, jsonify
import ollama
import json


print("Successfully imported!")

Starting imports...
Successfully imported!


In [24]:
# Load the dataset into a df
carsDF = pd.read_csv('vehicles.csv')

In [ ]:
# view dataset 
carsDF.head()

Data Cleaning and Preprocessing 

In [26]:
# Normalize text values to title case for consistency
carsDF['Make'] = carsDF['Make'].str.strip().str.upper()  # Remove extra spaces and convert to uppercase
carsDF['Model'] = carsDF['Model'].str.strip().str.upper()  # Remove extra spaces and convert to uppercase


In [ ]:
# Verify normalization
print(carsDF[['Make', 'Model']])

In [28]:
# Data Type Correction
# Convert numerical columns
carsDF['Year'] = carsDF['Year'].astype(int)
carsDF['SellingPrice'] = carsDF['SellingPrice'].astype(float)
carsDF['Doors'] = carsDF['Doors'].astype(int)


In [29]:
carsDF['PriceRange'] = pd.cut(carsDF['SellingPrice'], 
                              bins=[0, 10000, 20000, 30000, 50000, 100000],
                              labels=['Under 10K', '10K-20K', '20K-30K', '30K-50K', 'Above 50K'])


In [ ]:
app = Flask(__name__)

@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message')
    conversation_id = request.json.get('conversation_id')

    # Get response from Ollama model
    response = ollama.chat(
        model="your-llama-model",
        message=user_message,
        conversation_id=conversation_id
    )

    # Use LLM to extract user preferences
    user_preferences = extract_preferences(response['content'])
    filtered_results = search_cars(carsDF, **user_preferences)

    if filtered_results.empty:
        follow_up_message = "I couldn't find any cars matching your preferences. Would you like to modify your criteria?"
    else:
        top_cars = filtered_results.head(5)  # Get top 5 matches
        follow_up_message = format_car_suggestions(top_cars)

    # Respond with Ollama's response and filtered car suggestions
    return jsonify({"reply": follow_up_message, "conversation_id": response['conversation_id']})

# Run the server
if __name__ == "__main__":
    app.run(port=5000)

In [30]:
def search_cars(carsDF, car_type=None, year=None, make=None, model=None, model_number=None, stock=None, vin=None, certified=None):
    """
    Function to filter the cars DataFrame based on user-provided criteria.
    Arguments:
    - df: DataFrame containing car details.
    - car_type: 'New' or 'Used'
    - year: Year of manufacture (e.g., 2020)
    - make: Make of the car (e.g., 'Mazda')
    - model: Model of the car (e.g., 'CX-9')
    - model_number: Internal model code
    - stock: Dealer stock number
    - vin: Vehicle Identification Number
    - certified: Pre-owned certification status (True/False)
    
    Returns:
    - Filtered DataFrame based on provided criteria.
    """
    filtered_df = carsDF.copy()

    if car_type:
        filtered_df = filtered_df[filtered_df['Type'].str.upper() == car_type.upper()]
    if year:
        filtered_df = filtered_df[filtered_df['Year'] == year]
    if make:
        filtered_df = filtered_df[filtered_df['Make'].str.upper() == make.upper()]
    if model:
        filtered_df = filtered_df[filtered_df['Model'].str.upper() == model.upper()]
    if model_number:
        filtered_df = filtered_df[filtered_df['ModelNumber'].str.upper() == model_number.upper()]
    if stock:
        filtered_df = filtered_df[filtered_df['Stock'].str.upper() == stock.upper()]
    if vin:
        filtered_df = filtered_df[filtered_df['VIN'].str.upper() == vin.upper()]
    if certified is not None:  # Certified can be True or False
        filtered_df = filtered_df[filtered_df['Certified'] == certified]

    return filtered_df

# Example usage
user_search_results = search_cars(
    carsDF,
    car_type='Used',
    year=2019,
    make='Mazda',
    model='CX-9'
)

print(user_search_results[['Make', 'Model', 'Year', 'Certified', 'SellingPrice']])


    Make Model  Year  Certified  SellingPrice
0  MAZDA  CX-9  2019      False       18498.0


In [40]:
def chat_with_llama2(user_message, conversation_id):
    url = "http://localhost:11434/api/chat"
    headers = {"Content-Type": "application/json"}
    data = {
        "model": "llama2",
        "message": user_message,
        "conversation_id": conversation_id
    }

    response = requests.post(url, headers=headers, json=data)

    if response.status_code == 200:
        response_data = response.json()
        return response_data.get('content', "No response available")
    else:
        return f"Error: {response.status_code} - {response.text}"

# Example usage
conversation_id = "001"  # Unique ID to keep track of conversation context
user_message = "I need an affordable SUV with low mileage."
response = chat_with_llama2(user_message, conversation_id)
print(response)

NameError: name 'requests' is not defined